In [1]:
#CS5228 project

# data manipulation
import pandas as pd
import numpy as np
import math
import random
import pickle
import torch.optim as optim
from scipy import stats


# visualiation
import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline

# model training
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader


# classifiers
from sklearn.naive_bayes import GaussianNB # naive bayes
from sklearn.neighbors import KNeighborsClassifier # KNN
from sklearn.svm import SVC # SVM
from sklearn.linear_model import LogisticRegression # logistic regression
from sklearn.tree import DecisionTreeClassifier # decision Tree
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.feature_selection import RFE # for feature selection of LR
from sklearn.ensemble import BaggingClassifier 
from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier


# ignore warnings
import warnings
warnings.filterwarnings('ignore')


In [2]:
#load into dataframe
f_data = pd.read_csv('financial_data.csv', na_values=['?']) 
revealed = pd.read_csv('revealed_businesses.csv')
t_data = pd.read_csv('testing_data.csv', na_values=['?'])

display(f_data.head())
display(revealed.head())

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var56,Var57,Var58,Var59,Var60,Var61,Var62,Var63,Var64,Var65
0,18399,0.023954,0.150120,0.395670,3.63570,54.043,0.028822,0.031029,4.56831,1.01120,...,3871.001,0.011041,0.034914,0.98896,0.000000,9.5214,5.8248,34.713,10.5150,3.4752
1,15092,0.049699,0.065808,0.726800,12.94400,233.110,0.000000,0.063192,14.19601,0.89618,...,8751.901,0.059565,0.053189,0.93169,0.000000,5.0492,11.1520,24.784,14.7270,4.2204
2,19821,-0.356310,0.392880,0.158840,1.40430,-2.619,-0.085597,-0.356320,1.54531,0.92963,...,44.859,-0.172770,-0.586910,1.38330,0.000000,5.6112,15.7790,154.260,2.3662,2.0738
3,14171,0.001417,0.708110,-0.052312,0.88978,-31.198,0.269520,0.001407,0.41222,1.96540,...,-331.879,-0.000535,0.004820,0.99930,0.745480,17.1011,7.9482,88.147,4.1408,3.4021
4,12900,0.020041,0.346520,0.335930,2.76130,39.050,0.000000,0.020031,1.88591,1.29750,...,38170.001,0.212410,0.030652,0.80158,0.000862,9.7670,6.7570,53.651,6.8032,2.7412


,Var1,Var66
0,18399,0
1,19821,0
2,17769,0
3,19309,0
4,20728,0


In [21]:
#clean data
f_data_zeroes = f_data.replace(0, np.nan) # assume 0 values also means missing
t_data_zeroes = t_data.replace(0, np.nan)
#f_data_clean = f_data.replace(0, np.nan) # assume 0 values also means missing
#t_data_clean = t_data.replace(0, np.nan)
display(len(f_data_zeroes.index))
f_data_clean = f_data_zeroes.dropna(axis='columns',thresh=2)
display(f_data_zeroes.isna().sum().sort_values(ascending=False))
#display(t_data_zeroes.isna().sum())
#display(f_data_clean.isna().sum())



9003

Var38    4029
Var60    4029
Var7     3889
Var28    1203
Var43     888
Var23     855
Var22     665
Var48     563
Var46     549
Var21     504
Var61     499
Var50     227
Var65     222
Var54     201
Var55     201
Var29     201
Var49     194
Var42     184
Var33     103
Var58     101
Var53      90
Var34      89
Var24      87
Var35      85
Var40      77
Var64      47
Var62      47
Var45      47
Var36      46
Var63      46
         ... 
Var31      33
Var10      33
Var59      28
Var37      21
Var27      21
Var41      21
Var6       19
Var13      17
Var47      17
Var5       16
Var52      16
Var18      12
Var17      12
Var16      12
Var3       12
Var9       12
Var51      12
Var19       7
Var15       7
Var12       7
Var8        7
Var25       6
Var26       6
Var2        0
Var4        0
Var11       0
Var39       0
Var30       0
Var56       0
Var1        0
Length: 65, dtype: int64

In [102]:
#preprocess data
fdatan = f_data_clean.copy(deep=True)
normald = StandardScaler()
fdatan_1 = normald.fit_transform(fdatan.ix[:,fdatan.columns!="Var1"]) # normalize all variables except Var1
fdatan = pd.DataFrame(np.column_stack((fdatan["Var1"].values,fdatan_1)),
                      columns = fdatan.columns).set_index(fdatan.index)
tdatan = tdata.copy(deep=True)
normald = StandardScaler()
tdatan_1 = normald.fit_transform(tdatan.ix[:,tdatan.columns!="Var1"]) # normalize all variables except Var1
tdatan = pd.DataFrame(np.column_stack((tdatan["Var1"].values,tdatan_1)),
                      columns = tdatan.columns).set_index(tdatan.index)

fdatan.head()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var56,Var57,Var58,Var59,Var60,Var61,Var62,Var63,Var64,Var65
0,18399.0,-0.048725,-0.085746,0.051995,-0.012682,0.012603,0.036679,-0.057024,-0.019398,-0.104522,...,-0.048021,0.010500,0.004109,-0.017771,0.245694,-0.015621,-0.099369,-0.002156,0.042712,-0.079740
1,15092.0,-0.009047,-0.102278,0.117008,0.003792,0.014003,-0.016611,-0.007704,-0.005818,-0.118623,...,0.044772,0.011304,0.005021,-0.018535,-0.356855,-0.015736,-0.037961,-0.002513,0.179885,-0.078069
2,19821.0,-0.634787,-0.038144,0.005497,-0.016632,0.012159,0.015936,-0.650996,-0.023662,-0.114522,...,-0.120762,0.007453,-0.026925,-0.012515,0.403120,-0.015722,0.015376,0.002148,-0.222671,-0.082881
3,14171.0,-0.083459,0.023668,-0.035960,-0.017542,0.011936,0.080314,-0.102447,-0.025261,0.012458,...,-0.127924,0.010308,0.002607,-0.017634,-0.028036,-0.015427,-0.074892,-0.000232,-0.164877,-0.079903
4,12900.0,-0.054756,-0.047235,0.040266,-0.014230,0.012485,0.126787,-0.073888,-0.023182,-0.069423,...,0.604054,0.013837,0.003896,-0.020269,-0.036574,-0.015615,-0.088623,-0.001474,-0.078171,-0.081385


In [244]:
#merge dataframe
fdatan_merged = fdatan.merge(revealed, how="outer")
fdatan_train1 = fdatan.merge(revealed) # dataframe, whose bankruptcy status is known (0 = good standing, 1 = bankrupt)

display(revealed.head())
display(len(fdatan_train1[fdatan_train1.Var66==0].index))
len(fdatan_train1[fdatan_train1.Var66==1].index)

,Var1,Var66
0,18399,0
1,19821,0
2,17769,0
3,19309,0
4,20728,0


4709

170

In [96]:
#classifier here
x1 = fdatan_train1.drop(columns=['Var1', 'Var66']) # Var 1 is company ID, Var 66 is the status
y1 = fdatan_train1['Var66']
t1 = tdatan.drop(columns=['Var1'])
cID = tdata['Var1'].tolist() # use original values, since nothing is done in cID of tdata
clas = AdaBoostClassifier() # select classifier here
clas.fit(x1, y1)

# write results to predictions.csv
f = open('predictions.csv', 'w')
f.write('Business_ID,Is_Bankrupted\n')
for a,b in zip(cID, clas.predict(t1)):
    f.write(str(a))
    f.write(',')
    f.write(str(round(b)))
    f.write('\n')
f.close()

In [378]:
index_to_round = round(len(fdatan_train1.index)*0.75)
display("Dataset will be split on index: {}".format(index_to_round))

f_data_train_relevant = fdatan_train1[["Var1","Var54", "Var47","Var14","Var20","Var43","Var40","Var32","Var24","Var50",
                                      "Var57","Var6","Var55","Var37","Var29","Var5","Var41","Var65","Var51","Var52",
                                      "Var3", "Var66"]] 
for_training = f_data_train_relevant.iloc[:index_to_round, :]
for_testing = f_data_train_relevant.iloc[index_to_round:, :]

f_data_positives = for_training[for_training.Var66==0]
f_data_negatives = for_training[for_training.Var66==1]

x_pos = f_data_positives.drop(columns=['Var1', 'Var66'])
y_pos = f_data_positives.Var66
x_neg = f_data_negatives.drop(columns=['Var1', 'Var66'])
y_neg = f_data_negatives.Var66
x_training = for_training.drop(columns=['Var1', 'Var66'])
y_training = for_training.Var66


x_testing = for_testing.drop(columns=['Var1', 'Var66'])
y_testing = for_testing.Var66

display(len(x_pos.index))
display(len(x_neg.index))
display(f_data_train_relevant.head())

'Dataset will be split on index: 3659'

3532

127

,Var1,Var54,Var47,Var14,Var20,Var43,Var40,Var32,Var24,Var50,...,Var55,Var37,Var29,Var5,Var41,Var65,Var51,Var52,Var3,Var66
0,18399.0,-0.047036,-0.012879,-0.007115,0.018306,0.017040,0.012511,0.019150,0.018861,0.019250,...,-0.054590,-0.148320,-0.054708,-0.012682,-0.042860,-0.079740,-0.010554,-0.061522,-0.085746,0
1,19821.0,-0.048708,-0.015621,-0.013029,-0.015309,-0.017326,-0.009299,-0.014543,-0.014467,-0.018604,...,-0.056193,-0.406500,-0.059945,-0.016632,-0.040173,-0.082881,-0.014511,-0.013837,-0.038144,0
2,17769.0,-0.059277,-0.016913,-0.006789,0.016974,0.016361,-0.000545,0.017811,0.017901,0.016031,...,-0.066326,-0.339131,-0.069916,-0.018499,-0.060974,-0.084203,-0.016382,0.051017,0.026596,0
3,19309.0,-0.045815,-0.009409,-0.007249,0.017657,0.016336,0.011572,0.018497,0.018443,0.018565,...,-0.053419,0.162356,-0.056133,-0.009226,0.004689,-0.077292,-0.007091,-0.074460,-0.098662,0
4,20728.0,-0.051552,-0.015927,-0.006198,0.019316,0.013924,0.008353,0.020165,0.019511,0.013548,...,-0.057919,-0.120896,-0.060705,-0.016686,-0.060571,-0.080258,-0.014845,-0.018520,-0.033428,0


In [383]:
clf = KNeighborsClassifier()
clf.fit(x_training, y_training)
train_score = clf.score(x_training, y_training)
test_score = clf.score(x_testing, y_testing)
test_f1 = f1_score(y_testing, clf.predict(x_testing))
print('Train Score:{} Test Score:{}'.format(train_score, test_score))
print('Test F1:{}'.format(test_f1))

Train Score:0.9650177644165072 Test Score:0.9647540983606557
Test F1:0.0


In [384]:
# write results to predictions.csv
f = open('predictions.csv', 'w')
f.write('Business_ID,Is_Bankrupted\n')
for a,b in zip(cID, clf.predict(t1_test)):
    f.write(str(a))
    f.write(',')
    f.write(str(round(b)))
    f.write('\n')
f.close()

In [477]:
net = Net()
print(net)

Net(
  (fc1): Linear(in_features=20, out_features=100, bias=True)
  (do1): Dropout(p=0.2)
  (fc2): Linear(in_features=100, out_features=150, bias=True)
  (fc3): Linear(in_features=150, out_features=200, bias=True)
  (fc4): Linear(in_features=200, out_features=100, bias=True)
  (fc5): Linear(in_features=100, out_features=50, bias=True)
  (fc6): Linear(in_features=50, out_features=2, bias=True)
)


In [479]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)


cpu


In [480]:
for epoch in range(25):  # loop over the dataset multiple times

    running_loss = 0.0
    for i in range(0, len(x_neg.index)):
        x_train = torch.Tensor(x_neg.iloc[i].values).float()
        y_train = torch.tensor([1,0]).float()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(x_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()
        
        x_train = torch.Tensor(x_pos.iloc[np.random.randint(0, len(x_pos.index))].values).float()
        y_train = torch.tensor([0, 1]).float()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(x_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i == len(x_neg.index)-1:    # print every last mini-batch
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')


RuntimeError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [475]:
correct = 0
wrong = 0
correct_bankruptcy = 0
incorrect_bankruptcy = 0
for i in range(0, len(x_testing.index)):
    ref_x_test1 = torch.tensor(x_testing.iloc[i].values).float()
    ref_y_test1 = torch.tensor(y_testing.iloc[i]).float()
    outputs = net(ref_x_test1)
    if (round(ref_y_test1.item()) == round(outputs.item())):
        correct += 1
    else:
        wrong += 1
    if (round(ref_y_test1.item()) == 1 and round(outputs.item()) == 1):
        correct_bankruptcy +=1
    if (round(ref_y_test1.item()) == 1 and round(outputs.item()) == 0):
        incorrect_bankruptcy +=1
    #print('Bankruptcy guessed correctly!')
    #print('GroundTruth: {}'.format(ref_y_test1.item()))
    #print('Predicted: {}'.format(outputs.item()))
display('Number of correct: {}'.format(correct))
display('Number of wrong: {}'.format(wrong))
display('Number of correct bankruptcies: {}'.format(correct_bankruptcy))
display('Number of wrong bankruptcies: {}'.format(incorrect_bankruptcy))

'Number of correct: 765'

'Number of wrong: 455'

'Number of correct bankruptcies: 29'

'Number of wrong bankruptcies: 14'

In [452]:
t1_test = t1[["Var54", "Var47","Var14","Var20","Var43","Var40","Var32","Var24","Var50","Var57","Var6",
              "Var55","Var37","Var29","Var5","Var41","Var65","Var51","Var52","Var3"]]
f = open('predictions.csv', 'w')
f.write('Business_ID,Is_Bankrupted\n')
for i in range(0, len(t1_test.index)):
    x_test = torch.Tensor(t1_test.iloc[i].values).float()
    outputs = net(x_test)
    f.write(str(cID[i]))
    f.write(',')
    f.write(str(round(outputs.item())))
    f.write('\n')
f.close()

In [481]:
y_train

tensor([1., 0.])